In [1]:
# Dependencies and config file
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key, api_key2, api_key3, api_key4, api_key5, api_key6, api_key7
import openweathermapy as owm
from citipy import citipy
from random import randint
import numpy as np

In [2]:
# Randomly select 500 unique latitudes
latitude_list = []

for x in range(2500):
    latitude_list.append(np.random.uniform(low=-180.0, high=180, size=None))

# Convert list to series
latitude_list_series = pd.Series(latitude_list)

In [3]:
# Randomly select 500 unique longitudes
longitude_list = []

for x in range(2500):
    longitude_list.append(np.random.uniform(low=-90, high = 90))
    #print(longitude)

# Convert list to series
longitude_list_series = pd.Series(longitude_list)

In [4]:
# Concatentate the series into a df
lat_and_long = pd.concat([latitude_list_series, longitude_list_series], axis=1)
#lat_and_long.head(10)
#lat_and_long = lat_and_long.set_index(0)
lat_and_long.head(10)

,0,1
0,-102.071644,19.675111
1,-84.657578,-16.047003
2,23.855087,14.718499
3,144.589964,-53.279364
4,-24.561266,-3.712748
5,161.319685,-55.490209
6,-81.447503,-60.614207
7,-101.887053,-60.272463
8,104.962034,-23.134726
9,-65.194600,46.854129


In [5]:
# Create new empty columns and add to existing dataframe 
lat_and_long["City"] = ("")
lat_and_long["Country Code"] = ("")
lat_and_long["Temperature"] =("")
lat_and_long["Cloudiness"] =("")
lat_and_long["Windspeed"] =("")
lat_and_long.head(10)
# Rename columns
lat_and_long = lat_and_long.rename(columns={0: "Latitude", 1:"Longitude"})
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-102.071644,19.675111,,,,,
1,-84.657578,-16.047003,,,,,
2,23.855087,14.718499,,,,,
3,144.589964,-53.279364,,,,,
4,-24.561266,-3.712748,,,,,


In [6]:
# Blank column for holiding the city name
cities = []

#row_counter = 0
for index, row in lat_and_long.iterrows():
    x = row["Latitude"]
    y = row["Longitude"]
    city = citipy.nearest_city(x,y)
    lat_and_long.set_value(index, "City", citipy.nearest_city(x, y).city_name)
    lat_and_long.set_value(index, "Country Code", citipy.nearest_city(x, y).country_code)
    city_list = city.city_name
    cities.append(city_list)
#lat_and_long
#print(len(city_list))
#print(cities)

#cities = pd.Series(cities)

In [7]:
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-102.071644,19.675111,bredasdorp,za,,,
1,-84.657578,-16.047003,ushuaia,ar,,,
2,23.855087,14.718499,marzuq,ly,,,
3,144.589964,-53.279364,qaanaaq,gl,,,
4,-24.561266,-3.712748,jamestown,sh,,,


In [8]:
lat_and_long = lat_and_long.drop_duplicates(subset = ["City"], keep = "first")
lat_and_long = lat_and_long.reset_index(drop=True)
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-102.071644,19.675111,bredasdorp,za,,,
1,-84.657578,-16.047003,ushuaia,ar,,,
2,23.855087,14.718499,marzuq,ly,,,
3,144.589964,-53.279364,qaanaaq,gl,,,
4,-24.561266,-3.712748,jamestown,sh,,,


In [9]:
# Create empty list for storing city + country code together
city_country_list = []

# Iterate through df to populate the list of cities with their country code
for index, row in lat_and_long.iterrows():
    x = row["City"]
    y = row["Country Code"]
    city = str(x) + "," + str(y)
    #print(city)
    if city not in city_country_list:
        city_country_list.append(city) 
#print(city_country_list)

In [53]:
# Create settings dictionary and summary list
settings = {"units": "metric", "appid": api_key4}
summary = ["name", "sys.country", "coord.lon", "coord.lat", "main.temp", "wind.speed", "clouds.all"]

# Create a list to hold the dictionaries
meta_weather_summary = []
# loop through the city,country list to get the current weather
for x in city_country_list:
    print(f"Retrieving data for city:{x}.")
    try:
        current_weather = owm.get_current(x, **settings)
        weather_summary = current_weather(*summary)
        meta_weather_summary.append(weather_summary)
        #print(f"the weather is:{weather_summary}.")
        
    except: 
        print("No data found.")
print(meta_weather_summary)

Retrieving data for city:bredasdorp,za.
Retrieving data for city:ushuaia,ar.
Retrieving data for city:marzuq,ly.
No data found.
Retrieving data for city:qaanaaq,gl.
Retrieving data for city:jamestown,sh.
Retrieving data for city:illoqqortoormiut,gl.
No data found.
Retrieving data for city:east london,za.
Retrieving data for city:kruisfontein,za.
Retrieving data for city:clyde river,ca.
Retrieving data for city:cidreira,br.
Retrieving data for city:port alfred,za.
Retrieving data for city:aschaffenburg,de.
Retrieving data for city:dzilam gonzalez,mx.
Retrieving data for city:longyearbyen,sj.
Retrieving data for city:ribeira grande,pt.
Retrieving data for city:belushya guba,ru.
No data found.
Retrieving data for city:upernavik,gl.
Retrieving data for city:narsaq,gl.
Retrieving data for city:dikson,ru.
Retrieving data for city:kamenka,ru.
Retrieving data for city:albany,au.
Retrieving data for city:codrington,ag.
No data found.
Retrieving data for city:chinsali,zm.
Retrieving data for cit

Retrieving data for city:liwale,tz.
Retrieving data for city:langres,fr.
Retrieving data for city:bilma,ne.
Retrieving data for city:ashdod,il.
Retrieving data for city:jumla,np.
Retrieving data for city:oktyabrskoye,ru.
Retrieving data for city:ambovombe,mg.
Retrieving data for city:calama,cl.
Retrieving data for city:pavino,ru.
Retrieving data for city:nguru,ng.
Retrieving data for city:brae,gb.
Retrieving data for city:ust-kalmanka,ru.
Retrieving data for city:jizan,sa.
Retrieving data for city:hambantota,lk.
Retrieving data for city:tessalit,ml.
Retrieving data for city:key west,us.
Retrieving data for city:ponta do sol,cv.
Retrieving data for city:eyl,so.
Retrieving data for city:mocuba,mz.
Retrieving data for city:estelle,us.
Retrieving data for city:sandwick,gb.
Retrieving data for city:sivagiri,in.
Retrieving data for city:bonavista,ca.
Retrieving data for city:santo anastacio,br.
Retrieving data for city:talara,pe.
Retrieving data for city:balma,fr.
Retrieving data for city:to

Retrieving data for city:plouzane,fr.
Retrieving data for city:aflu,dz.
No data found.
Retrieving data for city:thunder bay,ca.
Retrieving data for city:mercedes,ar.
Retrieving data for city:bumba,cd.
Retrieving data for city:guarapari,br.
Retrieving data for city:san joaquin,bo.
Retrieving data for city:presidente epitacio,br.
Retrieving data for city:rockland,us.
Retrieving data for city:mahanoro,mg.
Retrieving data for city:oudtshoorn,za.
Retrieving data for city:old road,ag.
No data found.
Retrieving data for city:gobabis,na.
Retrieving data for city:hearst,ca.
Retrieving data for city:zaranj,af.
Retrieving data for city:san luis,gt.
Retrieving data for city:san pedro,bz.
Retrieving data for city:thompson,ca.
Retrieving data for city:bargal,so.
No data found.
Retrieving data for city:quatre cocos,mu.
Retrieving data for city:menfi,it.
Retrieving data for city:tanda,ci.
Retrieving data for city:verdalsora,no.
No data found.
Retrieving data for city:ferme-neuve,ca.
Retrieving data fo

In [54]:
meta_df = pd.DataFrame([meta_weather_summary])
meta_df.head()

,0,1,2,3,4,5,6,7,8,9,...,449,450,451,452,453,454,455,456,457,458
0,"(Bredasdorp, ZA, 20.04, -34.53, 27, 4.1, 24)","(Ushuaia, AR, -68.31, -54.81, 6.57, 6.2, 75)","(Qaanaaq, GL, -69.36, 77.48, -24.29, 3.31, 0)","(Jamestown, SH, -5.72, -15.94, 23.24, 0.86, 12)","(East London, ZA, 27.91, -33.02, 27, 5.7, 0)","(Kruisfontein, ZA, 24.73, -34, 28.06, 4.96, 0)","(Clyde River, CA, -68.59, 70.47, -20, 3.1, 75)","(Cidreira, BR, -50.22, -30.17, 21.66, 5.91, 80)","(Port Alfred, ZA, 26.89, -33.59, 26.34, 5.11, 0)","(Aschaffenburg, DE, 9.15, 49.97, 10.36, 3.1, 75)",...,"(Leh, IN, 77.58, 34.16, -15.39, 0.61, 76)","(Chuchkovo, RU, 41.45, 54.27, -0.37, 3.81, 44)","(Ancud, CL, -73.83, -41.87, 13.16, 3.46, 92)","(Chunhuhub, MX, -88.59, 19.59, 26.96, 2.56, 8)","(Kirkland Lake, CA, -80.03, 48.15, 1, 8.2, 75)","(Djougou, BJ, 1.67, 9.71, 35, 3.1, 0)","(Masoko, TZ, 33.75, -9.33, 24.71, 2.26, 32)","(Ginir, ET, 40.71, 7.14, 24.81, 2.61, 20)","(Gariaband, IN, 82.06, 20.63, 23.14, 2.06, 0)","(Velykoploske, UA, 29.68, 47.02, 8, 1.5, 0)"


In [51]:
views = {"summary": ["name", "sys.country", "coord.lon", "coord.lat", "main.temp", "wind.speed", "clouds.all"]}
settings = {"units": "metric", "appid": api_key2}
data = owm.get_current("London,UK", **settings)
data(*views["summary"])
print(data)

# return complete keys
city_dict = data.get_dict(views["summary"])
pd.DataFrame([city_dict])
#{'main.temp': 18.56, 'main.humidity': 63, 'main.pressure': 1011}

{'coord': {'lon': -0.13, 'lat': 51.51}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 13.54, 'pressure': 1003, 'humidity': 67, 'temp_min': 12, 'temp_max': 15}, 'visibility': 10000, 'wind': {'speed': 5.7, 'deg': 270}, 'clouds': {'all': 75}, 'dt': 1522162200, 'sys': {'type': 1, 'id': 5091, 'message': 0.0059, 'country': 'GB', 'sunrise': 1522129567, 'sunset': 1522175177}, 'id': 2643743, 'name': 'London', 'cod': 200}


,clouds.all,coord.lat,coord.lon,main.temp,name,sys.country,wind.speed
0,75,51.51,-0.13,13.54,London,GB,5.7


In [45]:
#import pandas as pd
#summary = ["name", "sys.country", "coord.lon", "coord.lat", "main.temp", "wind.speed", "clouds.all"]
#settings = {"units": "metric", "appid": api_key2}
#current_weather_test = owm.get_current('albany,au', **settings)
#print(current_weather_test)
#for x in current_weather:
#temp = current_weather_test("main.temp")
#print(temp)
    #print(temp)
#data = current_weather_test(*summary)
#df = pd.DataFrame.from_records(data)
lst = list(data)
print(lst)
#print(data)
#data = data.toFrame()
#data_df = pd.DataFrame.from_dict(data)
#df = pd.DataFrame(lst)

['Albany', 'AU', 117.88, -35.02, 16, 4.56, 0]


In [13]:
# Include a print log of each city as it is being processed with the city number, city name, and requested URL.
#print(f"Current weather object for {cities}: {data}.") 

In [14]:
# Save a CSV of all data retrieved

In [15]:
# Your objective is to build a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

In [16]:
# Build a scatter plot for each data type

#plt.scatter(df["temperature"].astype(int),
            #df["latitude"],
            #edgecolor="black", linewidths=1, marker="o",
            #alpha=0.8, label="?")

# Label the graph and set axis limits
#plt.title("")
#plt.ylabel("")
#plt.xlabel("")
#plt.grid(True)
#plt.xlim([, ])
#plt.ylim([, ])

# Save the figure as png image (for each scatter plot) 
#plt.savefig("UnemploymentRate_BankCount.png")

# Show plot
#plt.show()